# Opening a New Restaurant Near University of Toronto

_Joe and Anna are two entrepreneurs who are looking into opening a new restaurant in Toronto. They would like the restaurant to be aimed at university students since they have great ideas to provide a 'cool' environment for the youth. They already have the location of their new restaurant - a few blocks away from campus. However, they have not decided on what type of foods the new restaurant would serve (Italian, Mexican, or Chinese). They would prefer to enter business with as few direct competitors as possible. We will use Foursquare's API to help them analyze levels of competition across restaurant types so that they can decide on the cuisine of their new restaurant._

In [1]:
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.16.0-py_0 conda-forge

geographiclib- 100% |################################| Time: 0:00:00 394.73 kB/s
geopy-1.16.0-p 100% |################################| Time: 0:00:00 758.65 kB/s
Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py_0 conda-forge
    branca:  0.3.0-py_0 conda-forge
    folium:  0.5.0-py_0 conda-forge
    vincent: 0.4.4-py_1 conda-forge

branca-0.3.0-p 100% |################################| Time: 0:00:00 807.36 kB/s
vincent-0.4.4- 100% |################################| Time: 0:00:00 940.30 kB/s
altair-2.2.2-p 100% |###########################

__My Foursquare Credentials__

In [2]:
# The code was removed by Watson Studio for sharing.

Your credentails:
CLIENT_ID: TCIII2KKBPYXFSPVBJIBTCTGNI25UVKK5J5JRM0T4EFNWW00
CLIENT_SECRET:VPOHSUULNJORJFWFDWXS2HJWPSN51IMANSGJJQM4LVKQTXF5


## Let's start by converting University of Toronto's address to its latitude and longitude coordinates.

In [3]:
address = "27 King's College Cir, Toronto, Canada"

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/geopy/geocoders/osm.py:138: UserWarning: Using Nominatim with the default "geopy/1.16.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


43.6607225 -79.3959198095151


## Search for Nearby Restaurant Categories

**Italian**

In [4]:
search_query_italian = 'Italian'
radius = 800
print(search_query_italian + ' .... OK!')

Italian .... OK!


**Mexican**

In [5]:
search_query_mexican = 'Mexican'
radius = 800
print(search_query_mexican + ' .... OK!')

Mexican .... OK!


**Chinese**

In [6]:
search_query_chinese = 'Chinese'
radius = 800
print(search_query_chinese + ' .... OK!')

Chinese .... OK!


## Define URL

**URL Italian**

In [7]:
url_italian = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query_italian, radius, LIMIT)

**URL Mexican**

In [8]:
url_mexican = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query_mexican, radius, LIMIT)

**URL Chinese**

In [9]:
url_chinese = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query_chinese, radius, LIMIT)

## Examine Results

**Results Italian**

In [10]:
results_italian = requests.get(url_italian).json()
#results_italian

**Results Mexican**

In [11]:
results_mexican = requests.get(url_mexican).json()
#results_mexican

**Results Chinese**

In [12]:
results_chinese = requests.get(url_chinese).json()
#results_chinese

## Transform into Dataframe

**Italian Dataframe**

In [16]:
venues_italian = results_italian['response']['venues']

# tranform venues_italian into a dataframe
dataframe_italian = json_normalize(venues_italian)
#dataframe_italian.head()

**Mexican Dataframe**

In [17]:
venues_mexican = results_mexican['response']['venues']

# tranform venues_mexican into a dataframe
dataframe_mexican = json_normalize(venues_mexican)
#dataframe_mexican.head()

**Chinese Dataframe**

In [18]:
venues_chinese = results_chinese['response']['venues']

# tranform venues_chinese into a dataframe
dataframe_chinese = json_normalize(venues_chinese)
#dataframe_chinese.head()

## Define information of interest and filter dataframe

In [19]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

**Italian Filtered DF**

In [20]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns_italian = ['name', 'categories'] + [col for col in dataframe_italian.columns if col.startswith('location.')] + ['id']
dataframe_filtered_italian = dataframe_italian.loc[:, filtered_columns_italian]

# filter the category for each row
dataframe_filtered_italian['categories'] = dataframe_filtered_italian.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered_italian.columns = [column.split('.')[-1] for column in dataframe_filtered_italian.columns]
dataframe_filtered_italian = dataframe_filtered_italian.drop(dataframe_filtered_italian.index[[-1,1,2]])
#dataframe_filtered_italian

**Mexican Filtered DF**

In [21]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns_mexican = ['name', 'categories'] + [col for col in dataframe_mexican.columns if col.startswith('location.')] + ['id']
dataframe_filtered_mexican = dataframe_mexican.loc[:, filtered_columns_mexican]

# filter the category for each row
dataframe_filtered_mexican['categories'] = dataframe_filtered_mexican.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered_mexican.columns = [column.split('.')[-1] for column in dataframe_filtered_mexican.columns]
dataframe_filtered_mexican = dataframe_filtered_mexican.drop(dataframe_filtered_mexican.index[[3,4]])

#dataframe_filtered_mexican

**Chinese Filtered DF**

In [22]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns_chinese = ['name', 'categories'] + [col for col in dataframe_chinese.columns if col.startswith('location.')] + ['id']
dataframe_filtered_chinese = dataframe_chinese.loc[:, filtered_columns_chinese]

# filter the category for each row
dataframe_filtered_chinese['categories'] = dataframe_filtered_chinese.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered_chinese.columns = [column.split('.')[-1] for column in dataframe_filtered_chinese.columns]
dataframe_filtered_chinese = dataframe_filtered_chinese.drop(dataframe_filtered_chinese.index[[0,3,6,10,11,12,13]])

#dataframe_filtered_chinese

### Visualize Nearby Restaurants

_Now we will visualize the density of the three types of restaurants in an 800-meter radius from campus._

_The red mark represents University of Toronto, while blue, green, and purple marks signal Italian, Mexican, and Chinese restaurants, respectively_

In [23]:

venues_map = folium.Map(location=[latitude, longitude], zoom_start=14) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the University of Toronto
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='University of Toronto',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered_italian.lat, dataframe_filtered_italian.lng, dataframe_filtered_italian.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='navy',
        popup=label,
        fill = True,
        fill_color='navy',
        fill_opacity=0.6
    ).add_to(venues_map)

# add the Mexican restaurants as yellow circle markers
for lat, lng, label in zip(dataframe_filtered_mexican.lat, dataframe_filtered_mexican.lng, dataframe_filtered_mexican.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='green',
        popup=label,
        fill = True,
        fill_color='black',
        fill_opacity=0.6
    ).add_to(venues_map)

# add the Chinese restaurants as purple circle markers
for lat, lng, label in zip(dataframe_filtered_chinese.lat, dataframe_filtered_chinese.lng, dataframe_filtered_chinese.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='purple',
        popup=label,
        fill = True,
        fill_color='purple',
        fill_opacity=0.6
    ).add_to(venues_map)
    
# display map
venues_map

_As shown in the previous map, the highest level of competition falls in **Chinese** cuisine - by a clear margin. Thus, Joe and Anna will further consider only two types of cuisine for their new restaurant: **Italian and Mexican**_

**The following map shows locations only for Italian (blue) and Mexican (green) restaurants around University of Toronto (red).**

In [24]:

venues_map_1 = folium.Map(location=[latitude, longitude], zoom_start=14) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the University of Toronto
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='University of Toronto',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map_1)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered_italian.lat, dataframe_filtered_italian.lng, dataframe_filtered_italian.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='navy',
        popup=label,
        fill = True,
        fill_color='navy',
        fill_opacity=0.6
    ).add_to(venues_map_1)

# add the Mexican restaurants as yellow circle markers
for lat, lng, label in zip(dataframe_filtered_mexican.lat, dataframe_filtered_mexican.lng, dataframe_filtered_mexican.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='green',
        popup=label,
        fill = True,
        fill_color='black',
        fill_opacity=0.6
    ).add_to(venues_map_1)


    
# display map
venues_map_1

**Competition level is slightly higher, in terms of number of competitors, among Italian restaurants - compared to Mexican restaurants. However, Joe and Anna would also like to consider the average quality of the two types of restaurants (as measured by reviews) before making a decision about what foods their new restaurant will serve.**

_To do this, we will retrieve the venue 'name' and 'id' of each restaurant that we found on a previous query_

In [25]:
names_italian = dataframe_filtered_italian[['name','id']].reset_index()
print(names_italian)

   index                                 name                        id
0      0  Papa Ceo Fine Italian Foods & Pizza  4ad4c05ff964a52014f720e3
1      3                  john's italian cafe  53daae5b498e9c9597c19b23
2      4             The Fresh Italian Eatery  526fe29411d2aeb3803013b0
3      5                    The Fresh Italian  51bf3866498e55ee55df8db0


In [26]:
names_mexican = dataframe_filtered_mexican[['name','id']].reset_index()
print(names_mexican)

   index                       name                        id
0      0           Mexican Dry Food  4e57f282887710a8481de1f3
1      1     Chipotle Mexican Grill  571d1c1e498e166ab267532c
2      2  Don Taco Mexican Taqueria  51c5f916498e778c161a2420


### Italian Ratings

In [27]:
# The code was removed by Watson Studio for sharing.

In [28]:
# The code was removed by Watson Studio for sharing.

In [29]:
print(average_italian_ratings)    

5.5


### Mexican Ratings

In [30]:
# The code was removed by Watson Studio for sharing.

In [31]:
print(average_mexican_ratings)

7.4


**We can now see that, on average, Mexican restaurants around University of Toronto have a better rating than Italian restaurants in the area.**

_With our help, Joe and Anna have now decided what type of restaurant they will open nearby University of Toronto. They had two final options: Mexican and Italian. Joe and Anna's previous market study showed that the two cuisines are equally liked by young people (the target market of their new restaurant). There are about the same number of Italian and Mexican restaurants in the area. However, since Mexican restaurants already have good ratings given by customers, it will be more difficult to compete with them. For such reasons they will open a new Italian restaurant._